In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob,re

VIDEO_FRAMES_FOLDER = 'video_frames'


In [2]:
def extract_number(filename):
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    return 0
# numbers = re.compile(r'(\d+)')
# def numericalSort(value):
#     parts = numbers.split(value)
#     parts[1::2] = map(int, parts[1::2])
#     return parts

frames_dir = os.path.join(VIDEO_FRAMES_FOLDER, 'new_frames')
frames_array = glob.glob(frames_dir+'/*.png')
sorted_frames_array = sorted(frames_array, key=extract_number)
# print(sorted_frames_array)
frames_array = [cv2.imread(filename) for filename in sorted_frames_array]

In [3]:
def create_video_frames_folder(sub_folder_name):
    current_directory = os.getcwd()
    video_frames_directory = os.path.join(current_directory, VIDEO_FRAMES_FOLDER+'/'+sub_folder_name)
    if not os.path.exists(video_frames_directory):
        os.makedirs(video_frames_directory)

create_video_frames_folder('optical_flow')

In [4]:
def plot_optical_flow(frames, reference_frame_interval):
    st_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

    # lucas kanade optical flow params
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Initialize variables
    old_frame = frames[0]
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, **st_params)
    mask = np.zeros_like(old_frame)  # For drawing purposes

    reference_gray = old_gray
    p_reference = p0
    frame_index = 0
    color = np.random.randint(0, 255, (100, 3))
    # Loop over the frames
    for frame in frames[1:]:  # Start from the second frame
        if frame_index % reference_frame_interval == 0:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Calculate Optical Flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(reference_gray, frame_gray, p_reference, None, **lk_params)
            if p1 is not None:
                # Select good points
                good_new = p1[st == 1]
                good_old = p_reference[st == 1]
        
                # Draw the tracks
                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    a, b = new.ravel()
                    c, d = old.ravel()
                    mask = cv2.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
                    frame = cv2.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
        
                img = cv2.add(frame, mask)
                cv2.imshow('Frame', img)
                if cv2.waitKey(30) & 0xFF == ord('q'):
                    break
                
            # Updating previous frame and previous points
            reference_gray = frame_gray.copy()
            p_reference = good_new.reshape(-1,1,2)
        frame_index += 1
    
    optical_results_dir = os.path.join(VIDEO_FRAMES_FOLDER, 'optical_flow')
    cv2.imwrite(optical_results_dir+"/"+str(reference_frame_interval)+"_n.png", img)  
    cv2.imwrite(optical_results_dir+"/"+str(reference_frame_interval)+"_mask_n.png", mask)
    cv2.destroyAllWindows()

In [5]:
plot_optical_flow(frames_array, 1)

In [6]:
plot_optical_flow(frames_array, 11)

In [7]:
plot_optical_flow(frames_array, 31)